<h1 align=center><font size = 6> Coursera Capstone Project </font></h1>

<h2> Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <p><font size=4>Week 3 Submission</font></p>
     <ol>
         <li><a href="#w3_ans1">Neighborhoods in Toronto</a></li>
         <li><a href="#w3_ans2">Adding Geo-Codes to the Neighborhoods</a></li>
         <li><a href="#w3_ans3">Exploring the Neighborhoods</a></li>
    </ol>
    
</div>

<a id="w3_ans1"></a>
<h3>Part 1: Neighborhoods in Toronto</h3>

<ul> 1. Importing the libraries </ul>

In [18]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<ul>2. Scrapping the data</ul>

In [173]:
#creating link to the webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#request and store the data
data = requests.get(url).text

#create a BeautifulSoup object
soup = BeautifulSoup(data,"html5lib")

#find the html tabel
table = soup.find('table')

<ul> 3. Transforming the data </ul>

In [207]:
toronto_data = pd.DataFrame(columns =['PostalCode','Borough'])
for row in table.tbody.find_all("p"):
        for c1, c2 in zip(row.find_all("b"), row.find_all("span")):
            PostalCode = c1.text
            Borough = c2.text
            toronto_data = toronto_data.append({'PostalCode':PostalCode,'Borough':Borough}, ignore_index=True)

toronto_data = toronto_data[toronto_data['Borough']!="Not assigned"]
toronto_data[['Borough','Neighborhood','temp']] = toronto_data['Borough'].str.split('(', expand=True)
toronto_data['Neighborhood'] = toronto_data['Neighborhood'].str.replace(')',' ')
toronto_data['Neighborhood'] = toronto_data['Neighborhood']+' / '+toronto_data['temp'].str.replace(')','').fillna('')
toronto_data.drop(labels='temp', axis=1, inplace=True)
toronto_data['Neighborhood'] = toronto_data['Neighborhood'].str.replace(r'/ $','')
toronto_data['Neighborhood'] = toronto_data['Neighborhood'].str.replace(' /',',').str.strip()
toronto_data = toronto_data.reset_index(drop=True)

toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [208]:
print('#of rows in the dataframe: ', toronto_data.shape[0])

#of rows in the dataframe:  103


<a id="w3_ans2"></a>
<h3>Part 2: Adding Geo-Codes to the Neighborhoods</h3>

<ul> 1. Loading GeoSpatial Dataset </ul>

In [209]:
import io

geo_data_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
content = requests.get(geo_data_url).content
geo_data = pd.read_csv(io.StringIO(content.decode('utf-8')))
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<ul> 2. Adding Coordinates to Toronto Data </ul>

In [210]:
geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
toronto_data = pd.merge(toronto_data,geo_data,on='PostalCode',how='left')
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<ul> 3. Checking the result </ul>

In [213]:
print('NULL values after joining? :', toronto_data.isnull().values.any())
print('#of rows in the dataframe: ', toronto_data.shape[0])

NULL values after joining? : False
#of rows in the dataframe:  103


<a id="w3_ans3"></a>
<h3>Part 3: Exploring the Neighborhoods</h3>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [188]:
import os
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [2]:
address = 'Toronto City, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [3]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto